# Basic 3D Trajectories

This notebook demonstrates the generation, summarization, and visualization of basic 3D k-space trajectories using the `trajgen` library.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # For 3D plotting
from trajgen import KSpaceTrajectoryGenerator, Trajectory

# Ensure plots appear inline in the notebook
%matplotlib inline

## 1. 3D Stack-of-Spirals Trajectory

In [ ]:
# Instantiate generator for a 3D Stack-of-Spirals
n_stacks = 4
spirals_per_stack = 8
gen_sos = KSpaceTrajectoryGenerator(
    traj_type='stackofspirals',
    dim=3,
    fov=0.224,            # Field of View in meters (for in-plane)
    resolution=0.004,     # Resolution in meters (for in-plane)
    n_interleaves=spirals_per_stack, # Spirals per stack (Kz plane)
    n_stacks=n_stacks,             # Number of stacks (Kz planes)
    zmax_factor=0.9,             # Factor of k_max for z extent (0.9 * k_max_xy)
    turns=6
)

# Generate trajectory waveforms
# For stackofspirals, n_interleaves in generator is per Z-plane.
# The actual number of shots will be n_interleaves * n_stacks.
# The generate() method handles this internally and returns all shots.
kx_sos, ky_sos, kz_sos, gx_sos, gy_sos, gz_sos, t_sos = gen_sos.generate()

# kx, ky, kz are (total_shots, n_samples_per_shot)
# Combine all shots for the Trajectory object
kspace_sos_3d = np.stack([kx_sos.ravel(), ky_sos.ravel(), kz_sos.ravel()])
gradients_sos_3d = np.stack([gx_sos.ravel(), gy_sos.ravel(), gz_sos.ravel()])

# Create Trajectory object
traj_sos = Trajectory(
    name='3D Stack-of-Spirals Example',
    kspace_points_rad_per_m=kspace_sos_3d,
    gradient_waveforms_Tm=gradients_sos_3d,
    dt_seconds=gen_sos.dt,
    metadata={'gamma_Hz_per_T': gen_sos.gamma, 'generator_params': gen_sos.__dict__}
)

# Display trajectory summary
traj_sos.summary()

In [ ]:
# Plot a subset of the 3D Stack-of-Spirals k-space trajectory
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot every Nth point to keep the plot clean
step = 20 
ax.plot(kspace_sos_3d[0, ::step], kspace_sos_3d[1, ::step], kspace_sos_3d[2, ::step], '.', markersize=1)

# For better visualization, plot just a few full spirals from different stacks
# num_samples_per_spiral = gen_sos.n_samples
# for i in range(0, kx_sos.shape[0], spirals_per_stack // 2): # Plot a few spirals
#     ax.plot(kx_sos[i,:], ky_sos[i,:], kz_sos[i,:], '-')

ax.set_title('3D Stack-of-Spirals K-Space Trajectory (Subset)')
ax.set_xlabel('Kx (rad/m)')
ax.set_ylabel('Ky (rad/m)')
ax.set_zlabel('Kz (rad/m)')
plt.show()

## 2. 3D Radial Trajectory (Kooshball)

In [ ]:
# Instantiate generator for a 3D radial trajectory
gen_radial3d = KSpaceTrajectoryGenerator(
    traj_type='radial3d',
    dim=3,
    fov=0.200,
    resolution=0.005,
    n_interleaves=256,  # Number of radial spokes
    use_golden_angle=True # Use 3D golden angle (phyllotaxis-like) for spoke distribution
)

# Generate trajectory waveforms
kx_rad3d, ky_rad3d, kz_rad3d, gx_rad3d, gy_rad3d, gz_rad3d, t_rad3d = gen_radial3d.generate()

# Combine all interleaves (spokes)
kspace_rad3d_3d = np.stack([kx_rad3d.ravel(), ky_rad3d.ravel(), kz_rad3d.ravel()])
gradients_rad3d_3d = np.stack([gx_rad3d.ravel(), gy_rad3d.ravel(), gz_rad3d.ravel()])

# Create Trajectory object
traj_radial3d = Trajectory(
    name='3D Radial (Kooshball) Example',
    kspace_points_rad_per_m=kspace_rad3d_3d,
    gradient_waveforms_Tm=gradients_rad3d_3d,
    dt_seconds=gen_radial3d.dt,
    metadata={'gamma_Hz_per_T': gen_radial3d.gamma, 'generator_params': gen_radial3d.__dict__}
)

# Display trajectory summary
traj_radial3d.summary()

In [ ]:
# Plot a subset of the 3D Radial k-space trajectory
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot every Nth point from the ravelled data
step = 50 
ax.plot(kspace_rad3d_3d[0, ::step], kspace_rad3d_3d[1, ::step], kspace_rad3d_3d[2, ::step], '.', markersize=1)
ax.set_title('3D Radial K-Space Trajectory (Subset)')
ax.set_xlabel('Kx (rad/m)')
ax.set_ylabel('Ky (rad/m)')
ax.set_zlabel('Kz (rad/m)')
plt.show()

## 3. 3D EPI Trajectory

In [ ]:
# Define 3D EPI parameters
fov_readout = 0.256  # meters
res_readout = 0.004  # meters
fov_phase_y = 0.256  # meters
res_phase_y = 0.008  # meters (1/2 res of readout -> 32 lines for full FOV)
fov_phase_z = 0.200  # meters
res_phase_z = 0.016  # meters (1/4 res of readout -> 12-13 lines for full FOV)

num_encodes_y = int(round(fov_phase_y / res_phase_y))
num_encodes_z = int(round(fov_phase_z / res_phase_z))
total_epi_interleaves = num_encodes_y * num_encodes_z

print(f"EPI Y encodes: {num_encodes_y}, Z encodes: {num_encodes_z}, Total interleaves: {total_epi_interleaves}")

# Instantiate generator for a 3D EPI trajectory
gen_epi3d = KSpaceTrajectoryGenerator(
    traj_type='epi_3d',
    dim=3,
    fov=fov_readout,             # Readout FOV (x-direction)
    resolution=res_readout,      # Readout resolution (x-direction)
    epi_3d_fov_y=fov_phase_y,
    epi_3d_resolution_y=res_phase_y,
    epi_3d_fov_z=fov_phase_z,
    epi_3d_resolution_z=res_phase_z,
    n_interleaves=total_epi_interleaves # Total Ky-Kz planes
)

# Generate trajectory waveforms
kx_epi3d, ky_epi3d, kz_epi3d, gx_epi3d, gy_epi3d, gz_epi3d, t_epi3d = gen_epi3d.generate()

# Combine all interleaves (Ky-Kz planes)
kspace_epi3d_3d = np.stack([kx_epi3d.ravel(), ky_epi3d.ravel(), kz_epi3d.ravel()])
gradients_epi3d_3d = np.stack([gx_epi3d.ravel(), gy_epi3d.ravel(), gz_epi3d.ravel()])

# Create Trajectory object
traj_epi3d = Trajectory(
    name='3D EPI Example',
    kspace_points_rad_per_m=kspace_epi3d_3d,
    gradient_waveforms_Tm=gradients_epi3d_3d,
    dt_seconds=gen_epi3d.dt,
    metadata={'gamma_Hz_per_T': gen_epi3d.gamma, 'generator_params': gen_epi3d.__dict__}
)

# Display trajectory summary
traj_epi3d.summary()

In [ ]:
# Plot a subset of the 3D EPI k-space trajectory
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot every Nth point from the ravelled data to keep the plot somewhat clean
step = max(1, kspace_epi3d_3d.shape[1] // 2000) # Aim for around 2000 points
ax.plot(kspace_epi3d_3d[0, ::step], kspace_epi3d_3d[1, ::step], kspace_epi3d_3d[2, ::step], '.', markersize=1)
ax.set_title('3D EPI K-Space Trajectory (Subset)')
ax.set_xlabel('Kx (rad/m)')
ax.set_ylabel('Ky (rad/m)')
ax.set_zlabel('Kz (rad/m)')

# Set view angle for better visualization of EPI structure
ax.view_init(elev=20, azim=-45)
plt.show()

This notebook showcased how to generate, summarize, and visualize 3D trajectories like Stack-of-Spirals, 3D Radial (Kooshball), and 3D EPI. Due to the large number of points in 3D trajectories, only subsets are typically plotted for clarity.